In [1]:
from Bio import Entrez
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

pd.set_option('display.max_colwidth', 1000)

In [2]:
def search(query):
    Entrez.email = 'sxxize@gmail.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax='1000',
                            retmode='xml', 
                            term=query)
    results = Entrez.read(handle)
    return results

In [3]:
def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'sxxize@gmail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

In [19]:
def Author_list(papers):
    paper_author_lst=[i['MedlineCitation']['Article']['AuthorList'] for i in papers['PubmedArticle']]
    dfs=[pd.DataFrame(paper_author_lst[i]) for i in range(len(paper_author_lst))]
    names_dfs=pd.concat(dfs, axis=0, sort=True )
    author_count_df=names_dfs[['ForeName', 'LastName']]\
                    .groupby(['ForeName', 'LastName']).size()\
                    .reset_index(name='count').sort_values(by='count', ascending=False)
    top=author_count_df.head(10)
    google_url='https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q='
    name=top['ForeName']+' '+top['LastName']
    result=top.reset_index(drop=True).join(pd.DataFrame({'Google Scholar':[google_url+i for i in name.str.replace(' ', '+')+'+review&oq=']}))
    
    return result

In [17]:
def title_key(papers):
    titles=[i['MedlineCitation']['Article']['ArticleTitle'].lower()\
            .replace(',','').replace('.','').replace(':', '').replace('?','')\
            .replace('<sub>', '').replace('</sub>','').replace('<i>','').replace('</i>','')\
            .replace(search_word,'') for i in papers['PubmedArticle']]
    tfidf=TfidfVectorizer(ngram_range=(2,2),stop_words='english')
    X=tfidf.fit_transform(titles)
    tfidf_df=pd.DataFrame(X.todense(), columns=sorted(tfidf.vocabulary_))
    key_rank=tfidf_df.sum().sort_values(ascending=False)
    return key_rank


In [4]:
search_word='fuel cell'

In [5]:
results = search(search_word)
id_list = results['IdList']
papers = fetch_details(id_list)

# Key Word List

In [6]:
[str(i) for i in papers['PubmedArticle'][0]['MedlineCitation']['KeywordList'][0]]

['Carbon brush cylindrical microbial fuel cell',
 'Internal resistance',
 'Microbial fuel cell',
 'P. aeruginosa',
 'Polarization curve',
 'Power overshoot']

# Author List

In [16]:
result=Author_list(papers)
result

,ForeName,LastName,count,Google Scholar
0,Carlo,Santoro,19,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Carlo+Santoro+review&oq=
1,Plamen,Atanassov,16,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Plamen+Atanassov+review&oq=
2,Alexey,Serov,16,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Alexey+Serov+review&oq=
3,Hong,Liu,13,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Hong+Liu+review&oq=
4,Ioannis,Ieropoulos,13,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Ioannis+Ieropoulos+review&oq=
5,John,Greenman,12,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=John+Greenman+review&oq=
6,Wei,Zhou,9,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Wei+Zhou+review&oq=
7,Booki,Min,8,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Booki+Min+review&oq=
8,Zongping,Shao,8,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Zongping+Shao+review&oq=
9,S K,Kamarudin,8,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=S+K+Kamarudin+review&oq=


In [9]:
result['Google Scholar']

0         https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Carlo+Santoro+review&oq=
1      https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Plamen+Atanassov+review&oq=
2          https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Alexey+Serov+review&oq=
3              https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Hong+Liu+review&oq=
4    https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Ioannis+Ieropoulos+review&oq=
5         https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=John+Greenman+review&oq=
6              https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Wei+Zhou+review&oq=
7             https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Booki+Min+review&oq=
8         https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Zongping+Shao+review&oq=
9         https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=S+K+Kamarudin+review&oq=
Name: Google Scholar, dtype: object

In [10]:
#print(pd.DataFrame({'Google Scholar':[google_url+i for i in name.str.replace(' ', '+')+'+review&oq=']}))

# Key Words from Title

In [18]:
key_rank=title_key(papers)
key_rank[:20]

solid oxide                  14.281854
exchange membrane            10.643635
electricity generation       10.439862
proton exchange               9.669602
polymer electrolyte           8.670049
oxygen reduction              7.215937
microbial community           6.840695
performance microbial         6.831341
single chamber                6.477852
direct methanol               6.390811
wastewater treatment          6.200824
high performance              6.147326
air cathode                   6.129378
chamber microbial             6.058198
generation microbial          5.749626
electrolyte membrane          5.095756
temperature solid             5.031403
bioelectricity generation     4.862994
cathode microbial             4.856460
electricity production        4.520390
dtype: float64

LDA
word Vec
pyldavis

pyldavis

textrank
lexrank